In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [20]:
import numpy as np
import matplotlib.pyplot as plt

In [21]:
with open("poems.txt", "r", encoding="utf8") as f:
    text = f.read()

In [22]:
all_characters = set(text)

In [23]:
# Create a one hot encoder

def one_hot_encoder(encoded_text, num_uni_characters):
    
    
    # encoded_text --> batch of encoded text
    # num_uni_characters --> len(set(text))
    
    one_hot = np.zeros((encoded_text.size, num_uni_characters))
    one_hot = one_hot.astype(np.float32)
    one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()] = 1.0
    one_hot = one_hot.reshape((*encoded_text.shape, num_uni_characters))
    return one_hot

In [24]:
class CharModel(nn.Module):
    
    def __init__(self, all_chars, num_hidden=256, num_layers=4, dropout_porbability=0.5, use_gpu=False):
        super().__init__()
        
        self.dropout_porbability = dropout_porbability
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        self.use_gpu = use_gpu
        
        self.all_chars = all_chars
        self.decoder = dict(enumerate(all_chars))
        self.encoder = {char:ind for ind, char in self.decoder.items()}
        
        self.lstm = nn.LSTM(
            len(self.all_chars),
            num_hidden,
            num_layers,
            dropout=dropout_porbability,
            batch_first=True
        )
        
        self.dropout = nn.Dropout(dropout_porbability)
        
        self.fc_linear = nn.Linear(num_hidden, len(self.all_chars))
    
    
    def forward(self,x,hidden):
        
        lstm_output, hidden = self.lstm(x, hidden)
        
        drop_output = self.dropout(lstm_output)
        
        
        drop_output = drop_output.contiguous().view(-1, self.num_hidden)
        
        final_out = self.fc_linear(drop_output)
        
        return final_out, hidden
    
    def hidden_state(self, batch_size):
        if self.use_gpu:
            hidden = (
                torch.zeros(self.num_layers, batch_size, self.num_hidden).cuda(),
                torch.zeros(self.num_layers, batch_size, self.num_hidden).cuda()
            )
        else:
            hidden = (
                torch.zeros(self.num_layers, batch_size, self.num_hidden),
                torch.zeros(self.num_layers, batch_size, self.num_hidden)
            )
            
        return hidden

In [25]:
# MUST MATCH THE EXACT SAME SETTINGS AS MODEL USED DURING TRAINING!
model_name = "nepali_poem.net"

model = CharModel(
    all_chars=all_characters,
    num_hidden=512,
    num_layers=3,
    dropout_porbability=0.5,
    use_gpu=False,
)

model.load_state_dict(torch.load(model_name, map_location=lambda storage, loc: storage))
model.eval()

CharModel(
  (lstm): LSTM(225, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_linear): Linear(in_features=512, out_features=225, bias=True)
)

In [29]:
def predict_next_char(model, char, hidden=None, k=1):
    
    encoded_text = model.encoder[char]
    
    encoded_text = np.array([[encoded_text]])
    
    encoded_text = one_hot_encoder(encoded_text, len(model.all_chars))
    
    inputs = torch.from_numpy(encoded_text)
    
    if model.use_gpu:
        inputs = inputs.cuda()
    
    hidden = tuple([state.data for state in hidden])
    
    lstm_out , hidden = model(inputs, hidden)
    
    probs = F.softmax(lstm_out, dim=1).data
    
    if model.use_gpu:
        probs = probs.cpu()
        
    probs, index_positions = probs.topk(k)
    
    index_positions = index_positions.numpy().squeeze()
    
    probs = probs.numpy().flatten()
    
    probs = probs/probs.sum()
    
    char = np.random.choice(index_positions, p=probs)
    
    return model.decoder[char], hidden

In [30]:
def generate_text(model, size, seed="The", k=1):
    
    if model.use_gpu:
        model.cuda()
    else:
        model.cpu()
        
    model.eval()
    
    output_chars = [c for c in seed]
    
    hidden = model.hidden_state(1)
    
    for char in seed:
        char, hidden = predict_next_char(model, char, hidden, k=k)
    
    output_chars.append(char)
    
    for i in range(size):
        char, hidden = predict_next_char(model, output_chars[-1], hidden, k=k)
        
        output_chars.append(char)
    
    
    return ''.join(output_chars)

In [31]:
print(generate_text(model, 1000, seed = "मेरो ", k=3))

मेरो tञь’ू–२hञdसÞьञh२ऊमtèस१पऩhmdसEप–२hञtसπफьऩdसÞьञtoऊणञपसÞऩदस[द२ऊस१२ь´ऩьEप–[द२ьञdसÞьञh२सÞьञh२२tसÞपhञ–२स[द२ऊमtèसÞऩदसणd=ऩसर––२स२स[स[द२ьञdसÞьञh२mdस१ьंtषद२tऩस–२स१t…ऊसणपv’ूऩस–m१ूमhसÞऩदस२स[द२ऊमhसÞऩदस–२स२hञdसÞьञh२सÞऩदसणपv’ूऩसर––२hञtसπvBt२tसतtमmसÞऩदसणtv१ьEसर––––ÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔ–१पञьऐसÞьञh२–सिस१t२ьतस१mьञऊ–२hञdस’hःmdस२tऩьEh–[ьऐ१ूमhस२मtèस२tऐtसञद’दऩपसÞञь’ूऩ–२स[स१२ь२सÞऩदसणd–२सm१ूmdस१पऩь’ञसÞपञtऩdसÞt[ьञ२tसÞऩदस–mhणऊसÞऩदस[सणपऩьEस–२hञdस१tv´सतtvँhmdसEसरस––२hञdस’hःmdस२ऩ२tसÞऩदस–२स[स२hञdसπ२tmdस२tऐtस–२hञूसÞьञदऐस२hञdस२पMप२t–१२ऐसतtvँьEप–[द२ऊस[स[द२ьञdस२ऩmdस१ьंtषद२tऩऊसतdmhञस–m१ञऊसऩूस१२ь´hञस–२स[द२ьञtस१ьं[ऩь[ьञ[tसणपऩьEपस–[द२ьञdस१ьंञस[द२ьञdसÞьञh२सÞt=ऩhसर–––ÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔÔ–[द२ऊमtèसÞьञ[ьऐhmस१t२ऩь[–सिस२ऩdलसÞdञऊमस–२सÞऩदस१tv´सÞऩदस[सणd=ऩस–२hञdस१ьÞञьः२tसÞऩदस१पऩhञस–२स[स१२ь´दञणhEप–[द२ьञdसmंद[t२t…द–२tऩंस१२ऐmdसÞьञ[दतद२ьत–[द२ऊमtèसÞऩदस१t…सणdस––२सm१ूमhस१पऩhञ–mपऩूसÞьञh२सतtvँhmdसEसर––[ञस२स[स१पऩhmdस